In [1]:
import numba
from numba import jit, int32, prange, vectorize, float64, cuda
import numpy as np
import math

In [2]:
# Function that calculates the value of Pi using monte-carlo method
# Minimal use of Numpy
def native_python_monte_carlo(n):
    within_circle = 0
    
    for i in range(int(n)):
        x = np.random.random()
        y = np.random.random()
        
        if x**2 + y**2 <= 1.0:
            within_circle += 1
        
    return 4.0 * within_circle / n

In [3]:
%%time
print(native_python_monte_carlo(10000000))

3.14163
CPU times: user 11 s, sys: 10.6 ms, total: 11 s
Wall time: 11 s


In [4]:
# Function that calculates the value of Pi using monte-carlo method
# Leveraged numpy as far as possible

def numpy_python_monte_carlo(n):
    within_circle = 0
    
    x = np.random.random(int(n))
    y = np.random.random(int(n))
        
    within_circle = np.sum( (x**2 + y**2) <= 1.0 )
        
    return 4.0 * within_circle / n

In [5]:
%%time
print(numpy_python_monte_carlo(10000000))

3.1411872
CPU times: user 168 ms, sys: 56 ms, total: 224 ms
Wall time: 223 ms


### Use @jit on both functions and see what is the speedup achieved (ignore the first run)
* Native python :
* Numpy :

* Natve python with Numba :
* Numpy wuth Numba: 

In [6]:
@jit
def native_python_monte_carlo(n):
    within_circle = 0
    
    for i in range(int(n)):
        x = np.random.random()
        y = np.random.random()
        
        if x**2 + y**2 <= 1.0:
           within_circle += 1
        
    return 4.0 * within_circle / n

In [9]:
%%time
print(native_python_monte_carlo(10000000))

3.140952
CPU times: user 97.3 ms, sys: 0 ns, total: 97.3 ms
Wall time: 96.8 ms


In [16]:
@jit
def numpy_python_monte_carlo(n):
    within_circle = 0
    
    x = np.random.random(int(n))
    y = np.random.random(int(n))
        
    within_circle = np.sum( (x**2 + y**2) <= 1.0 )
        
    return 4.0 * within_circle / n

In [19]:
%%time
print(numpy_python_monte_carlo(10000000))

3.1421672
CPU times: user 124 ms, sys: 49.5 ms, total: 173 ms
Wall time: 172 ms


### What is the difference If we use eager compilation (ignore first run)?

* Native python with Numba (Lazy):
* Native python with Numba (Eager):

In [20]:
@jit(numba.float32(int32))
def native_python_monte_carlo_eager(n):
    within_circle = 0
    
    for i in range(int(n)):
        x = np.random.random()
        y = np.random.random()
        
        if x**2 + y**2 <= 1.0:
            within_circle += 1
        
    return 4.0 * within_circle / n

In [23]:
%%time
print(native_python_monte_carlo_eager(10000000))

3.142198324203491
CPU times: user 101 ms, sys: 0 ns, total: 101 ms
Wall time: 100 ms


In [25]:
%%time
@jit
def native_python_monte_carlo_lazy(n):
    within_circle = 0
    
    for i in range(int(n)):
        x = np.random.random()
        y = np.random.random()
        
        if x**2 + y**2 <= 1.0:
            within_circle += 1
        
    return 4.0 * within_circle / n

CPU times: user 368 µs, sys: 36 µs, total: 404 µs
Wall time: 411 µs


In [30]:
%%time
print(native_python_monte_carlo_lazy(10000000))

3.1421032
CPU times: user 97.1 ms, sys: 0 ns, total: 97.1 ms
Wall time: 96.7 ms


### What is the difference with and without automatic parallelization (ignore the first run)?
hint: use *numba.prange* instead of *range*
* With parallel:
* Without parallel:

In [31]:
@jit(parallel=True)
def native_python_monte_carlo_parallel(n):
    within_circle = 0
    
    for i in numba.prange(int(n)):
        x = np.random.random()
        y = np.random.random()
        
        if x**2 + y**2 <= 1.0:
            within_circle += 1
        
    return 4.0 * within_circle / n

In [34]:
%%time
print(native_python_monte_carlo_parallel(10000000))

3.1412404
CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 56 ms


### Offload the computation to a GPU
* Assumptions:
    * N <= 512
* Hints:
    * Launch one block with threads <= 512

In [35]:
@cuda.jit
def native_python_monte_kernel(array_a, array_b, array_c):
    # Thread id in a 1D block
    tx = cuda.threadIdx.x
 
    if tx < array_a.size:  # Check array boundaries
        array_c[tx] = 1 if array_a[tx]**2 + array_b[tx]**2 <= 1.0  else 0

In [37]:
n = 10000
blocks = 2
threads = 256
array_a = np.random.random(n)
array_b = np.random.random(n)
array_c = np.zeros(n)

In [38]:
native_python_monte_kernel[blocks, threads](array_a, array_b, array_c)
print(4.0 * np.sum(array_c) / n)

/scratch/vp91/Training-Venv/parallelpython/lib/python3.10/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 2 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


CudaSupportError: Error at driver init: Call to cuInit results in CUDA_ERROR_NO_DEVICE (100)